In [ ]:
!pip install pyspark py4j

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('weather_data_pyspark').getOrCreate()
df = spark.read.csv("/content/Weather Training Data.csv", header=True, inferSchema=True)
print(df)

DataFrame[row ID: string, Location: string, MinTemp: double, MaxTemp: double, Rainfall: double, Evaporation: double, Sunshine: double, WindGustDir: string, WindGustSpeed: int, WindDir9am: string, WindDir3pm: string, WindSpeed9am: int, WindSpeed3pm: int, Humidity9am: int, Humidity3pm: int, Pressure9am: double, Pressure3pm: double, Cloud9am: int, Cloud3pm: int, Temp9am: double, Temp3pm: double, RainToday: string, RainTomorrow: int]
